In [20]:
import yaml
import numpy as np
import pylab as plt
from math import *
from types import *
from operator import itemgetter
from inspect import getargspec
from iminuit import Minuit,util
%matplotlib inline

In [21]:
def f(x, a):
    return np.tanh(x*a)

def g(x, b):
    return np.power(x,b)

def h(x, c, d):
    return x*c+d*np.cos(x)

def j(x, e, f):
    return e*np.power(x,f)

In [22]:
x_arr = np.linspace(-10,10)

#plt.plot(x_arr,f(x_arr,2)*g(x_arr,3));
#plt.plot(x_arr,f(x_arr,2)*h(x_arr,3,2));
#plt.plot(x_arr,j(x_arr,2,1)*h(x_arr,3,2));

In [23]:
class loglike(object):
    """ class to define the loglike. this takes a function f as argument,
        extracts its signature, checks if the first argument is 'x' and 
        returns the loglike as a function of the remaining arguments """
    
    def __init__(self, data, func):
        self.data = data
        self.func = func
        self.arguments = {}
        
    def get_args(self):
        signature = getargspec(self.func)[0]
        if signature[0] == 'x':
            for arg in signature[1:]:
                self.arguments[arg] = 1. #fictious value
                # next: extract info from config file and create two dictionaries,
                # one for fixed variables in the fit -> add 'fixed_<variable>':True to dict
                # one with free variables and assign initial guess value or fixed value, parsed from config file
            return self.arguments
        else:
            raise ValueError, 'first argument should be "x"'
    
    def __call__(self, *freepars):
        arguments = self.get_args().keys()
        for i,freepar in enumerate(arguments):
            self.get_args()[freepar] = freepars[i]
        
        is_tuple = isinstance(freepars, TupleType)
        if is_tuple:
            return sum((self.func(self.data[0],*freepars)- self.data[1])**2)
        else:
            return sum((self.func(self.data[0], freepars)- self.data[1])**2)

In [24]:
class MinuitFitter(object):
    def __init__(self, loglike):
        self.loglike = loglike
        global_loglike = loglike
        self.settings = {'errordef':0.5,'print_level':1,'pedantic':False}
        
    def sync(self):
        for key,val in self.loglike.get_args().items():
            self.settings[key] = 1.
            self.settings['error_%s'%key] = 0.01
        return self.settings
        

    def fit(self, **kwargs):
        settings = self.sync()
        freepars = self.loglike.get_args().keys()
        strargs = ", ".join(freepars)
        fit_func = eval("lambda {args} : global_loglike({args})".format(args=strargs))
        mm = Minuit(fit_func, **settings)
        if 'tol' in kwargs: 
            mm.tol = kwargs['tol']
        #return fit_func
        fitresult = mm.migrad()
        return fitresult



In [25]:
c_true = 3
d_true = 2
y_arr = h(x_arr, c_true, d_true)
LL = loglike((x_arr, y_arr),h)

global global_loglike
global_loglike = LL

m = MinuitFitter(LL)
m.sync()
print m.settings
m.fit()

{'pedantic': False, 'd': 1.0, 'c': 1.0, 'print_level': 1, 'errordef': 0.5, 'error_d': 0.01, 'error_c': 0.01}


FCN = 1.04831374749e-14 
 TOTAL NCALL = 28 
 NCALLS = 28 
 
 
 EDM = 1.04831371523e-14 
 GOAL EDM = 5e-06 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,c,3.000000e+00,1.697749e-02,0.000000e+00,0.000000e+00,,,
2,d,2.000000e+00,1.379879e-01,0.000000e+00,0.000000e+00,,,


({'hesse_failed': False, 'has_reached_call_limit': False, 'has_accurate_covar': True, 'has_posdef_covar': True, 'up': 0.5, 'edm': 1.0483137152280821e-14, 'is_valid': True, 'is_above_max_edm': False, 'has_covariance': True, 'has_made_posdef_covar': False, 'has_valid_parameters': True, 'fval': 1.048313747491732e-14, 'nfcn': 28},
 [{'is_const': False, 'name': 'c', 'has_limits': False, 'value': 2.9999999975622353, 'number': 0, 'has_lower_limit': False, 'upper_limit': 0.0, 'lower_limit': 0.0, 'has_upper_limit': False, 'error': 0.016977493752581632, 'is_fixed': False},
  {'is_const': False, 'name': 'd', 'has_limits': False, 'value': 1.9999999974230729, 'number': 1, 'has_lower_limit': False, 'upper_limit': 0.0, 'lower_limit': 0.0, 'has_upper_limit': False, 'error': 0.13798790064135202, 'is_fixed': False}])

In [26]:
c_true = 3
d_true = 2
y_arr = h(x_arr, c_true, d_true)
settings = {'c': 1.0, 'd': 1.0, 'print_level': 1, 'pedantic': False, 'error_c': 0.01, 'error_d': 0.01, 'errordef': 0.5}
def proxy_func(c, d):
    return np.sum((h(x_arr, c, d)-y_arr)**2)
m = Minuit(proxy_func, **settings)
m.migrad()

FCN = 3.76340321183e-15 
 TOTAL NCALL = 28 
 NCALLS = 28 
 
 
 EDM = 3.76340314841e-15 
 GOAL EDM = 5e-06 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,c,3.000000e+00,1.697749e-02,0.000000e+00,0.000000e+00,,,
2,d,2.000000e+00,1.379879e-01,0.000000e+00,0.000000e+00,,,


({'hesse_failed': False, 'has_reached_call_limit': False, 'has_accurate_covar': True, 'has_posdef_covar': True, 'up': 0.5, 'edm': 3.763403148411169e-15, 'is_valid': True, 'is_above_max_edm': False, 'has_covariance': True, 'has_made_posdef_covar': False, 'has_valid_parameters': True, 'fval': 3.763403211834842e-15, 'nfcn': 28},
 [{'is_const': False, 'name': 'c', 'has_limits': False, 'value': 2.9999999985286268, 'number': 0, 'has_lower_limit': False, 'upper_limit': 0.0, 'lower_limit': 0.0, 'has_upper_limit': False, 'error': 0.016977493752926568, 'is_fixed': False},
  {'is_const': False, 'name': 'd', 'has_limits': False, 'value': 2.000000000548371, 'number': 1, 'has_lower_limit': False, 'upper_limit': 0.0, 'lower_limit': 0.0, 'has_upper_limit': False, 'error': 0.1379879006416467, 'is_fixed': False}])